# App citation (IV) 제작

### 1. 기업간 adj matrix 제작
- (patent_id -> number)를 (gvkey -> gvkey)로 바꾸기

### 2. weight로 사용할 similarity
- app cite한 기업 사이 technological similarity (proximity) 계산 
- 각 기업별 patent class 비율 vector 구해서 correlation 보면 됨 

### 3. weight sum
- 걍 weight sum해서 firm level 변수 제작

### 1. 기업간 adj matrix 제작
- 특정 산업에 해당하는 patent selection (patent_class, app_citation_byfirm join)  
__=> patent_id, number, year__  
- patent_id -> assignee -> gvkey (patent_assignee join, assignee_gvkey join)   
__=> gvkey, number, year__
- number -> raw_assignee -> assignee -> gvkey (app_info join, assignee_raw join, assignee_gvkey join)  
__=> gvkey, gvkey_cited, year__
- year 기준으로 subset, 각각에 대해 adj matrix 제작 (여긴 R 사용)

In [4]:
import pandas as pd
import numpy as np
import os

os.chdir('E:/apps/db_table')

__by firm 부터 ㄱㄱ__  
__1. patent selection__

In [2]:
bio = pd.read_csv('patent_class_BIO.csv')
ee = pd.read_csv('patent_class_EE&IT.csv')

bio = bio.loc[:,['patent_id','year']]
ee = ee.loc[:,['patent_id','year']]

print len(bio), len(ee)

139224 850774


In [3]:
df = pd.read_csv('app_citation_byfirm.csv')

bio = bio.merge(df,how='left',on='patent_id')
ee = ee.merge(df,how='left',on='patent_id')

bio.dropna(how='any',inplace=True)  #drop patents which don't cite apps
ee.dropna(how='any',inplace=True)
print len(bio), len(ee)

1061284 2680241


In [4]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__2. patent_id -> gvkey__

In [5]:
df = pd.read_csv('patent_assignee.csv')

bio = bio.merge(df, how='left', on='patent_id')
ee = ee.merge(df, how='left', on='patent_id')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee')
ee = ee.merge(df1, how='left', on='assignee')

bio = bio.loc[:,['gvkey','number','year']]
ee = ee.loc[:,['gvkey','number','year']]

len(bio), len(ee)

(1088254, 2760338)

In [6]:
bio.dropna(how='any',inplace=True)  #drop patents which don't have gvkey of assignee
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(527529, 1484488)

In [7]:
ee.iloc[0:10,:]

,gvkey,number,year
0,1976.0,20030085039,2005
1,1976.0,20030185099,2005
3,104604.0,20020070991,2005
4,5606.0,20030082841,2005
5,5606.0,20010015744,2005
6,61552.0,20030052944,2005
7,61552.0,20030016279,2005
8,61552.0,20020191061,2005
9,61552.0,20020186284,2005
10,61552.0,20020145650,2005


In [8]:
ee.isnull().any()

gvkey     False
number    False
year      False
dtype: bool

In [9]:
bio.isnull().any()

gvkey     False
number    False
year      False
dtype: bool

In [10]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__3. number -> gvkey_cited__

In [11]:
bio = pd.read_csv('bio_temp.csv')
ee = pd.read_csv('ee_temp.csv')

C:\Users\DW\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df = pd.read_csv('app_info.csv')
df = df.loc[:,['number','raw_assignee']]
df = df.loc[df['raw_assignee']!='private',:]  #selecting non-private apps
df['number'] = map(str,df['number'])

bio['number'] = map(str,bio['number'])
ee['number'] = map(str,ee['number'])

bio = bio.merge(df, how='left', on='number')
ee = ee.merge(df, how='left', on='number')

len(bio), len(ee)

(536106, 1493918)

In [13]:
bio.dropna(how='any',inplace=True)  #drop patents which cited assignee-missing application
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(224617, 643978)

In [14]:
ee.iloc[0:10,:]

,gvkey,number,year,raw_assignee
0,1976.0,20030085039,2005,"Baker Hughes, Inc."
1,1976.0,20030185099,2005,Baker Hughes Incorporated
2,104604.0,20020070991,2005,Seiko Epson Corporation
9,61552.0,20020145650,2005,Eastman Kodak Company
10,61552.0,20020105567,2005,BROTHER KOGYO KABUSHIKI KAISHA
39,7343.0,20030096497,2005,"Micron Technology, Inc."
53,7343.0,20020163828,2005,Coatue Corporation
74,7343.0,20020190350,2005,ARIZONA BOARD OF REGENTS
80,7772.0,20020089067,2005,Loctite Corporation
82,7114.0,20020024119,2005,Kabushiki Kaisha Toshiba


In [15]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

In [16]:
df = pd.read_csv('assignee_raw.csv')

bio = bio.merge(df, how='left', on='raw_assignee')
ee = ee.merge(df, how='left', on='raw_assignee')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee', suffixes=('', '_cited'))
ee = ee.merge(df1, how='left', on='assignee', suffixes=('', '_cited'))

bio = bio.loc[:,['gvkey','gvkey_cited','year']]
ee = ee.loc[:,['gvkey','gvkey_cited','year']]

len(bio), len(ee)

(224953, 644369)

In [17]:
bio.dropna(how='any',inplace=True)  #drop patents which cited gvkey-missing application
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(100703, 332001)

In [18]:
ee.isnull().any()

gvkey          False
gvkey_cited    False
year           False
dtype: bool

In [19]:
bio.isnull().any()

gvkey          False
gvkey_cited    False
year           False
dtype: bool

In [ ]:
bio['gvkey'] = map(int,bio['gvkey'])
ee['gvkey'] = map(int,ee['gvkey'])

bio.to_csv('E:/apps/app_cit_edge_BIO.csv',index=False)
ee.to_csv('E:/apps/app_cit_edge_EE&IT.csv',index=False)

__r 써서 마무리__

### 2. weight로 사용할 similarity (tech proximity)
__2.1 기업별 patent class vector 제작__
- 기업별 patent class 파악 (patent_class, patent_assignee, gvkey join)  
__=> gvkey, uspc, year__  
- 기업, 연도 레벨로 patent class aggregate(count, groupby gvkey, year, uspc)  
__=>gvkey, year, uspc, count__
- matrix로 제작 (uspc X gvkey pivot), count를 비율로 변경 

__2.2 기업간 proximity matrix 제작__
- 전체 기업 중 appcit_matrix에 있는 기업만 고르기 (appcit_matrix의 0번째 col, class_vector join)
- 함수 사용 matrix 제작

__1. 기업별 patent class 파악__

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('E:/apps/db_table')

In [3]:
bio = pd.read_csv('patent_class_BIO.csv')
ee = pd.read_csv('patent_class_EE&IT.csv')

bio = bio.loc[:,['patent_id','uspc','year']]
ee = ee.loc[:,['patent_id','uspc','year']]

print len(bio), len(ee)

139224 850774


In [4]:
df = pd.read_csv('patent_assignee.csv')

bio = bio.merge(df,how='left',on='patent_id')
ee = ee.merge(df,how='left',on='patent_id')

bio = bio.loc[:,['assignee','uspc','year']]
ee = ee.loc[:,['assignee','uspc','year']]

bio.dropna(how='any',inplace=True)  #drop patents which don't have assignee
ee.dropna(how='any',inplace=True)
print len(bio), len(ee)

116260 783601


In [5]:
df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df,how='left',on='assignee')
ee = ee.merge(df1,how='left',on='assignee')

bio = bio.loc[:,['gvkey','uspc','year']]
ee = ee.loc[:,['gvkey','uspc','year']]

bio.dropna(how='any',inplace=True)  #drop patents which don't have assignee
ee.dropna(how='any',inplace=True)
print len(bio), len(ee)

54112 554481


In [6]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__2. firm-year level aggregation__

In [7]:
bio['count'] = ''
ee['count'] = ''

bio = bio.groupby(['gvkey','year','uspc'], as_index=False).agg('count')
ee = ee.groupby(['gvkey','year','uspc'], as_index=False).agg('count')

print len(bio), len(ee)

5480 56897


In [8]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__3. matrix로 변경__

In [2]:
bio = pd.read_csv('bio_temp.csv')
ee = pd.read_csv('ee_temp.csv')

In [3]:
def get_ratio(row):
    tot = sum(row)
    row = map(lambda x: round(float(x)/tot,3), row)    

    return row

def pivot(df,y):
    df = df.loc[df['year']==y,:]
    df = pd.pivot_table(df,index='gvkey',columns='uspc', values='count', fill_value=0)
    df = df.apply(get_ratio, axis=1,reduce = False)
    
    return df

In [4]:
os.chdir('E:/apps/appcit measure')

for i in range(2005,2013):
    bdf = pivot(bio,i)
    fname = 'class_vector_bio_' + str(i) +'.csv'
    bdf.to_csv(fname)
    
    edf = pivot(ee,i)
    fname = 'class_vector_EE&IT_' + str(i) +'.csv'
    edf.to_csv(fname)

__4. proximity 계산__

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import math
from math import sqrt

os.chdir('E:/apps/appcit measure')

In [2]:
def prox_numer(l1,l2):
    cnt = 0.0
    for i in xrange(len(l1)):
        if l1[i]!=0 and l2[i]!=0:
            w= float(l1[i]) * l2[i]
            cnt += w
    
    return cnt 

def prox_denom(l1,l2):
    a = sqrt(sum(map(lambda x: x**2, l1)))
    b = sqrt(sum(map(lambda x: x**2, l2)))
    
    return a*b 

def proximity(l1,l2):
    return round(prox_numer(l1,l2) / prox_denom(l1,l2),3)

def get_prox(l1,arr):
    sim = []
    for row in arr:
        sim.append(proximity(l1,row))
        
    return sim

In [3]:
def final(ind,y):
    gvkey = 'appcit_matrix_' + ind + '_' + str(y) +'.csv'
    clsvec = 'class_vector_' + ind + '_' + str(y) +'.csv'

    df = pd.read_csv(gvkey).iloc[:,0].to_frame()
    df1 = pd.read_csv(clsvec)
    
    df = df.merge(df1,how='left',on='gvkey')
    arr = df.values

    header = arr[:,0].tolist()
    header = ['gvkey'] + map(int,header)

    arr = np.delete(arr,0,1)
    
    fname = 'proximity_' + ind + '_' + str(y) +'.csv'
    with open(fname,'wb') as f:
        writer = csv.writer(f)
        writer.writerow(header)

        i=1  
        for row in arr:        
            line = get_prox(row,arr)
            line = [header[i]] + line             
            writer.writerow(line)
            i += 1

In [4]:
for y in range(2005,2013):
    final('bio',y)
    final('EE&IT',y)

### 3. weight sum
- matrix끼리 곱하기

In [2]:
def get_appcit(ind,y):
    cit = 'appcit_matrix_' + ind + '_' + str(y) +'.csv'
    prox = 'proximity_' + ind + '_' + str(y) +'.csv'
    
    cit = pd.read_csv(cit)
    prox = pd.read_csv(prox)

    df = cit * prox

    df['app_cit'] = 0.0
    
    for i in range(0,len(df)):
        df.loc[i,'gvkey'] = int(sqrt(df.loc[i,'gvkey']))
        df.loc[i,'app_cit'] = sum(df.iloc[i,1:]) - df.iloc[i,i+1]
        
    df['year'] = y
    df = df.loc[:,['gvkey','year','app_cit']]
    
    fname = 'appcit_' + ind + '_' + str(y) +'.csv'
    df.to_csv(fname,index=False)

In [3]:
for y in range(2005,2013):
    get_appcit('bio',y)
    get_appcit('EE&IT',y)

- 하나로 만들기

In [17]:
bio = pd.read_csv('appcit_bio_2005.csv')
ee = pd.read_csv('appcit_EE&IT_2005.csv')

for y in range(2006,2013):
    df = pd.read_csv('appcit_bio_' + str(y) +'.csv')
    bio = pd.concat([bio,df],ignore_index=True)
    
    df = pd.read_csv('appcit_EE&IT_' + str(y) +'.csv')
    ee = pd.concat([ee,df],ignore_index=True)
    
bio.sort_values(by=['gvkey','year'],inplace=True)
bio.reset_index(drop=True, inplace=True)
ee.sort_values(by=['gvkey','year'],inplace=True)
ee.reset_index(drop=True,inplace=True)

In [19]:
bio.to_csv('appcit_bio.csv',index=False)
ee.to_csv('appcit_EE&IT.csv',index=False)